In [5]:
from utils import *
from create_dataset import get_filtered_dataset
from descision_trees import DecisionTree
from create_dataset import save_filtered_dataset_for_neurons

In [6]:
'''big_dataset = pd.read_csv(f"data/neuron_datasets/logic_L{layer}.csv")
save_filtered_dataset_for_neurons(big_dataset=big_dataset,dataset_name="logic_small", layer = layer, neurons = [0, 1, 2, 3, 4, 421])
big_dataset = pd.read_csv(f"data/neuron_datasets/logic_no_softmax_L{layer}.csv")
save_filtered_dataset_for_neurons(big_dataset=big_dataset,dataset_name="logic_no_softmax_small", layer = layer, neurons = [0, 1, 2, 3, 4, 421])'''

'big_dataset = pd.read_csv(f"data/neuron_datasets/logic_L{layer}.csv")\nsave_filtered_dataset_for_neurons(big_dataset=big_dataset,dataset_name="logic_small", layer = layer, neurons = [0, 1, 2, 3, 4, 421])\nbig_dataset = pd.read_csv(f"data/neuron_datasets/logic_no_softmax_L{layer}.csv")\nsave_filtered_dataset_for_neurons(big_dataset=big_dataset,dataset_name="logic_no_softmax_small", layer = layer, neurons = [0, 1, 2, 3, 4, 421])'

In [7]:
from neuron_predictor import NeuronPredictor

# import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor  
import math

import numpy as np
from utils import probe_directions_list, tuple_to_label
import os

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import joblib

def get_variable_names():
    variable_names = []
    for probe_name in ["linear", "flipped", "placed"]:
        for row in range(8):
            for col in range(8):
                for option in probe_directions_list[probe_name]:
                    label = tuple_to_label((row, col))
                    variable_names.append(f"{label} {option}")
    return variable_names

def tree_to_cnf(tree, feature_names=None):
    recurse_count = 0
    def recurse(node, path):
        nonlocal recurse_count
        recurse_count += 1
        if tree.feature[node] != -2:  # not a leaf
            feature = feature_names[tree.feature[node]] if feature_names is not None else f"feature_{tree.feature[node]}"

            if feature[3:].startswith("not"):
                left_feature = feature[:3] + feature[7:]
                right_feature = feature
            else:
                left_feature = feature[:3] + "not " + feature[3:]
                right_feature = feature
            
            left_path = path + [f"{left_feature}"]
            right_path = path + [f"{right_feature}"]
            
            yield from recurse(tree.children_left[node], left_path)
            yield from recurse(tree.children_right[node], right_path)
        else:  # leaf
            predicted_value = tree.value[node][0, 0]
            yield f"({' AND '.join(path)} => {predicted_value:.4f})"

    return list(recurse(0, [])), recurse_count

def get_accuracy(y_pred, y_test):
    correct = 0
    for i in range(len(y_test)):
        if (y_pred[i] > 0 and y_test[i] > 0) or (y_pred[i] <= 0 and y_test[i] <= 0):
            correct += 1
    return correct / len(y_test)

def get_f1(y_pred, y_test):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(y_test)):
        if y_pred[i] > 0 and y_test[i] > 0:
            tp += 1
        elif y_pred[i] <= 0 and y_test[i] <= 0:
            tn += 1
        elif y_pred[i] > 0 and y_test[i] <= 0:
            fp += 1
        elif y_pred[i] <= 0 and y_test[i] > 0:
            fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

EPSILON = 0.000001

def scorefunction(depth, accuracy):
    return math.log(accuracy + EPSILON, 2) * 2**depth

class DecisionTree(NeuronPredictor):
    def __init__(self, layer, neuron):
        self.column_names_input = get_variable_names()
        self.neuron = neuron
        self.layer = layer

    def predict(self, X):
        return self.regressor.predict(X)

    def fit(self, X, y, **kwargs):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        top_score = -math.inf
        # if max_depth is not None:
        #    self.max_depth = max_depth
        # else:
        #     self.max_depth = 3
        self.max_depth = 3
        self.regressor = DecisionTreeRegressor(random_state = 0, **kwargs)  
        self.regressor.fit(X_train, y_train)
        # I think high max_depth will be the main limmiting time factor
        """for max_depth in range(1, 7):
            regressor = DecisionTreeRegressor(random_state = 0, max_depth=max_depth)  
            regressor.fit(X_train, y_train)
            y_pred = regressor.predict(X_test)
            # Get the f1 score using a library function
            f1 = f1_score(y_test > 0, y_pred > 0)
            # f1 = get_f1(y_pred, y_test)
            score = scorefunction(max_depth, f1)
            if score > top_score:
                top_score = score
                self.regressor = regressor
                self.max_depth = max_depth
                if f1 >= 0.95:
                    break"""

    def get_clean_format(self, column_names):
        if column_names is None:
            column_names = self.column_names_input
        cnf_rules = tree_to_cnf(self.regressor.tree_, feature_names=column_names)
        return cnf_rules

    def get_sparcity(self):
        return 1 / (self.max_depth * 2**self.max_depth)

    def load(self, layer, neuron):
        # Save the decision tree to a file
        file_path = f"neuron_predictors/decision_trees/decision_tree_L{layer}_N{neuron}.joblib"
        if os.path.exists(file_path):
            self.regressor = joblib.load(file_path)
            self.max_depth = self.regressor.get_depth()
            self.layer = layer
            self.neuron = neuron
            return True
        return False

    def save(self):
        file_path = f"neuron_predictors/decision_trees/decision_tree_L{self.layer}_N{self.neuron}.joblib"#
        # Save the decision tree to a file
        joblib.dump(self.regressor, file_path)

In [8]:
def weighted_f1_score(y_pred, y_test):
    """
    Calculate the f1 score using vecorized operations.
    The inputs are continuous, so we need to threshold them at zero.
    - for the true positives and false positves I don’t add 1 to the true / false negatives score but N * C where
    - N is the neuron activation
    - And C is sum(neuron activations) / count(neuron activations)
    """
    C = np.sum(y_test[y_test > 0]) / len(y_test[y_test > 0])
    tp = np.sum(np.logical_and(y_pred > 0, y_test > 0) * (y_test * C))
    tn = np.sum(np.logical_and(y_pred <= 0, y_test <= 0))
    fp = np.sum(np.logical_and(y_pred > 0, y_test <= 0))
    fn = np.sum(np.logical_and(y_pred <= 0, y_test > 0) * (y_test * C))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [9]:
layer = 1
neuron = 2
'''np.random.seed(42)
small_dataset_no_softmax = pd.read_csv(f"data/neuron_datasets/logic_no_softmax_small_L{layer}.csv")
dataset_no_softmax = get_filtered_dataset(small_dataset_no_softmax, layer, neuron, size = 100000, overfitting_strength=None)
small_dataset_softmax = pd.read_csv(f"data/neuron_datasets/logic_small_L{layer}.csv")
np.random.seed(42)
dataset_softmax = get_filtered_dataset(small_dataset_softmax, layer, neuron, size = 100000, overfitting_strength=None)
dataset_no_softmax.columns = [f"{col}_no_softmax" for col in dataset_no_softmax.columns]'''
# merge the two datasets the key is the index or neuron activation
small_dataset_softmax = pd.read_csv(f"data/neuron_datasets/logic_small_L{layer}.csv")
dataset = get_filtered_dataset(small_dataset_softmax, layer, neuron, size = 100000, overfitting_strength=None)

In [10]:
# dataset = pd.merge(dataset_no_softmax, dataset_softmax, on="neuron activation", suffixes=('_no_softmax', '_softmax'))
# dataset = pd.concat([dataset_no_softmax, dataset_softmax], axis=1)
'''dataset = dataset_no_softmax
# apply softmax to the dataset
# create the input tensor 
in_tensor = t.stack([t.from_numpy(dataset["G4 yours_no_softmax"].values), t.from_numpy(dataset["G4 mine_no_softmax"].values), t.from_numpy(dataset["G4 empty_no_softmax"].values)], dim=1)
in_tensor = t.softmax(in_tensor, dim=1)
in_tensor = in_tensor[:, 0]
in_tensor = in_tensor.numpy()
dataset["G4 yours"] = in_tensor

columns = dataset.columns.tolist()
columns = columns[:-2] + [columns[-1]] + [columns[-2]]
dataset = dataset[columns]
# small_dataset_no_softmax["L1_N2"]
# small_dataset_softmax["L1_N2"]
# dataset[["neuron activation", "neuron activation_no_softmax"]]'''


'dataset = dataset_no_softmax\n# apply softmax to the dataset\n# create the input tensor \nin_tensor = t.stack([t.from_numpy(dataset["G4 yours_no_softmax"].values), t.from_numpy(dataset["G4 mine_no_softmax"].values), t.from_numpy(dataset["G4 empty_no_softmax"].values)], dim=1)\nin_tensor = t.softmax(in_tensor, dim=1)\nin_tensor = in_tensor[:, 0]\nin_tensor = in_tensor.numpy()\ndataset["G4 yours"] = in_tensor\n\ncolumns = dataset.columns.tolist()\ncolumns = columns[:-2] + [columns[-1]] + [columns[-2]]\ndataset = dataset[columns]\n# small_dataset_no_softmax["L1_N2"]\n# small_dataset_softmax["L1_N2"]\n# dataset[["neuron activation", "neuron activation_no_softmax"]]'

In [11]:
'''dataset.loc[dataset["neuron activation"] <= 0, "neuron activation"] = -1.0
dataset.loc[dataset["neuron activation"] > 0, "neuron activation"] = 1.0
print(dataset[dataset["neuron activation"] == 1].shape)
print(dataset[dataset["neuron activation"] == -1].shape)'''

'dataset.loc[dataset["neuron activation"] <= 0, "neuron activation"] = -1.0\ndataset.loc[dataset["neuron activation"] > 0, "neuron activation"] = 1.0\nprint(dataset[dataset["neuron activation"] == 1].shape)\nprint(dataset[dataset["neuron activation"] == -1].shape)'

In [12]:
dataset

,A0 empty,A0 yours,A0 mine,A1 empty,A1 yours,A1 mine,A2 empty,A2 yours,A2 mine,A3 empty,...,H3 not_placed,H4 placed,H4 not_placed,H5 placed,H5 not_placed,H6 placed,H6 not_placed,H7 placed,H7 not_placed,neuron activation
0,9.999995e-01,5.515071e-08,4.272128e-07,1.000000e+00,9.077296e-11,8.722876e-11,1.000000e+00,7.453085e-09,2.429726e-09,6.348039e-08,...,1.0,5.822188e-09,1.0,3.686365e-09,1.0,7.276853e-09,1.0,8.235369e-08,1.0,-0.014301
1,2.899494e-09,1.056404e-08,1.000000e+00,4.430894e-08,2.618030e-10,1.000000e+00,7.120000e-07,6.170320e-07,9.999987e-01,9.999999e-01,...,1.0,7.972168e-11,1.0,3.191337e-12,1.0,1.735693e-12,1.0,2.145722e-10,1.0,-0.143468
2,5.638960e-07,3.548654e-08,9.999994e-01,3.992198e-05,2.661746e-06,9.999574e-01,5.932582e-09,3.731384e-09,1.000000e+00,9.999715e-01,...,1.0,2.239009e-09,1.0,5.198147e-11,1.0,1.686210e-08,1.0,6.121551e-11,1.0,-0.111531
3,1.000000e+00,6.313376e-11,1.511813e-10,1.000000e+00,3.278454e-12,1.073964e-10,1.000000e+00,1.828848e-10,5.434128e-12,1.353134e-08,...,1.0,7.686816e-12,1.0,5.903713e-11,1.0,2.764745e-12,1.0,4.050180e-12,1.0,-0.009904
4,9.999999e-01,5.878171e-08,7.293071e-08,1.000000e+00,3.777942e-08,3.436203e-08,9.999999e-01,9.457933e-09,1.016457e-07,9.999994e-01,...,1.0,9.180273e-10,1.0,3.863364e-10,1.0,1.910162e-11,1.0,2.458694e-10,1.0,-0.144023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.000000e+00,1.135108e-11,2.602362e-10,1.000000e+00,5.925223e-12,3.116869e-12,1.000000e+00,1.928693e-13,1.400643e-11,1.454319e-09,...,1.0,7.411708e-12,1.0,5.147491e-12,1.0,3.414533e-11,1.0,1.116944e-10,1.0,-0.012884
99996,9.357086e-07,9.999981e-01,9.741315e-07,1.000000e+00,1.258117e-11,3.253899e-10,5.547406e-06,6.420652e-07,9.999938e-01,9.999989e-01,...,1.0,1.752856e-12,1.0,1.770287e-12,1.0,3.864053e-12,1.0,8.121384e-13,1.0,-0.031531
99997,7.648549e-06,1.416934e-07,9.999923e-01,7.337126e-10,5.717401e-09,1.000000e+00,9.999999e-01,7.683366e-09,1.156250e-07,1.000000e+00,...,1.0,9.429070e-11,1.0,3.397192e-09,1.0,4.869549e-09,1.0,1.963614e-09,1.0,-0.037659
99998,1.000000e+00,5.689933e-12,6.320590e-12,1.000000e+00,9.955564e-12,5.324504e-11,1.000000e+00,4.138698e-12,5.479517e-12,1.000000e+00,...,1.0,9.349378e-11,1.0,4.897777e-11,1.0,2.787989e-10,1.0,4.021123e-12,1.0,-0.001246


In [21]:
def new_weighted_f1_score(y, y_pred):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(y)):
        score = max(y[i], y_pred[i])
        if y_pred[i] > 0 and y[i] > 0:
            tp += score
        elif y_pred[i] <= 0 and y[i] <= 0:
            tn += 1
        elif y_pred[i] > 0 and y[i] <= 0:
            fp += score
        elif y_pred[i] <= 0 and y[i] > 0:
            fn += score
    precision = tp / (tp + fp + EPSILON)
    recall = tp / (tp + fn + EPSILON)
    f1 = 2 * (precision * recall) / (precision + recall + EPSILON)
    return f1.item()

# print(new_weighted_f1_score(y_test, y_pred_test))

In [22]:
def evaluate(y_pred, y, threshold):
    y_pred_binary = y_pred > threshold
    y_binary = y > threshold
    return precision_score(y_binary, y_pred_binary), recall_score(y_binary, y_pred_binary), f1_score(y_binary, y_pred_binary)

def pred_manual(formula : list[list[str]], X, threshold = 0.9999839067):
    y_pred = np.ones((X.shape[0])) * -1
    for i in range(len(y_pred)):
        for rule in formula:
            if all([X[feature][i] > threshold for feature in rule]):
                # print(i)
                y_pred[i] = 0.3
                break
    return y_pred

column_names = dataset.columns
column_names_input = column_names[:-1]
column_names_output = [column_names[-1]]
X = dataset[column_names_input].astype(float) 
y = dataset[column_names_output].astype(float)
y = y.to_numpy()
y_pred = pred_manual([["G4 yours", "E6 placed"],
                      ["G4 yours", "H3 placed"],
                      ["G4 yours", "D1 placed"],
                      ["G4 yours", "C0 placed"],
                      ["G4 yours", "E2 placed"],
                      ["G4 yours", "G1 placed"],
                      ["G4 yours", "G2 placed"],
                      ["G4 yours", "G2 placed"],
                      ["G4 yours", "G3 placed"],
                      ["G4 yours", "H4 placed"],
                      ["G4 yours", "A4 placed"],
                      ["G4 yours", "B4 placed"],
                      ["G4 yours", "C4 placed"],
                      ["G4 yours", "D7 placed"],], X, 0.99)[:, None]

'''y_pred = pred_manual([["G4 yours", "E6 placed"],
                      ["G4 yours", "H3 placed"],
                      ["G4 yours", "D1 placed"],], X, 0.90)[:, None]'''
# y_pred = pred_manual([["G4 yours"]], X)[:, None]
print(evaluate(y_pred, y, 0))
print(new_weighted_f1_score(y, y_pred))
compare = np.concatenate([y, y_pred], axis=1)
print(compare[:20])
# X["G4 yours"][1]
# print(X.iloc[5][X.iloc[5] > 0.9])
X["G4 yours"][5]

(0.08795257930150593, 0.19985438660356752, 0.1221492935810435)
0.2556439211104805
[[-0.16023673 -1.        ]
 [-0.1352806  -1.        ]
 [-0.04884819 -1.        ]
 [-0.15180685 -1.        ]
 [-0.16963921 -1.        ]
 [-0.05052318  0.3       ]
 [-0.04871121 -1.        ]
 [-0.11899538  0.3       ]
 [-0.0341941  -1.        ]
 [-0.16973707  0.3       ]
 [-0.16640826 -1.        ]
 [-0.06518679 -1.        ]
 [-0.1323253  -1.        ]
 [-0.10429908 -1.        ]
 [-0.0113814  -1.        ]
 [-0.1587878  -1.        ]
 [-0.10987219 -1.        ]
 [-0.04079624 -1.        ]
 [-0.16786101 -1.        ]
 [-0.10607117 -1.        ]]


0.9999943

In [23]:
layer = 1
neuron = 421
small_dataset_softmax = pd.read_csv(f"data/neuron_datasets/logic_small_L{layer}.csv")
dataset = get_filtered_dataset(small_dataset_softmax, layer, neuron, size = 100000, overfitting_strength=None)
dataset_columns_new = dataset.columns.tolist()
dataset_columns_new = [col_name for col_name in dataset_columns_new if col_name[3:6] != "not"]
dataset = dataset[dataset_columns_new]

In [24]:
descision_tree = DecisionTree(layer, neuron)
column_names = dataset.columns
column_names_input = column_names[:-1]
column_names_output = [column_names[-1]]

X = dataset[column_names_input].astype(float)
y = dataset[column_names_output].astype(float)
# turn into numpy arrays
X = X.to_numpy()
y = y.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# descision_tree.fit(X_train, y_train, min_impurity_decrease=0.0001)
descision_tree.fit(X_train, y_train, max_depth=3, min_impurity_decrease=0.001)
descision_tree.predict(X)
y_pred_train = descision_tree.predict(X_train)
y_pred_test = descision_tree.predict(X_test)

In [25]:
descision_tree.get_clean_format(column_names_input)

(['(B4 not placed AND A4 not placed AND B4 not empty => -0.1152)',
  '(B4 not placed AND A4 not placed AND B4 empty => -0.0430)',
  '(B4 not placed AND A4 placed AND B4 not flipped => -0.0643)',
  '(B4 not placed AND A4 placed AND B4 flipped => 1.7161)',
  '(B4 placed AND C4 not empty AND B2 not flipped => 1.9735)',
  '(B4 placed AND C4 not empty AND B2 flipped => 1.3467)',
  '(B4 placed AND C4 empty => -0.1112)'],
 13)

In [26]:
layer = 1
small_dataset_softmax = pd.read_csv(f"data/neuron_datasets/logic_small_L{layer}.csv")

In [34]:
# Create a datafram with columns neuron, min_impurity_decrease, f1_weighted, f1, sparcity and fill it with the results
# results = pd.DataFrame(columns=["neuron", "min_impurity_decrease", "f1_weighted", "f1", "sparcity"])
results = {
    "neuron": [],
    "min_impurity_decrease": [],
    "f1_weighted": [],
    "f1": [],
    "CNF size" : [],
}

def get_data(layer, neuron):
    small_dataset_softmax = pd.read_csv(f"data/neuron_datasets/logic_small_L{layer}.csv")
    dataset = get_filtered_dataset(small_dataset_softmax, layer, neuron, size = 100000, overfitting_strength=None)
    dataset_columns_new = dataset.columns.tolist()

    dataset_columns_new = [col_name for col_name in dataset_columns_new if col_name[3:6] != "not"]
    dataset = dataset[dataset_columns_new]

    column_names = dataset.columns
    column_names_input = column_names[:-1]
    column_names_output = [column_names[-1]]

    X = dataset[column_names_input].astype(float)
    y = dataset[column_names_output].astype(float)
    # turn into numpy arrays
    X = X.to_numpy()
    y = y.to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def train_decision_tree(X_train, y_train, layer, neuron, **kwargs) -> DecisionTree:
    descision_tree = DecisionTree(layer, neuron)
    descision_tree.fit(X_train, y_train, **kwargs)
    return descision_tree

def evaluate_decision_tree(descision_tree : DecisionTree, X_test, y_test):
    y_pred_test = descision_tree.predict(X_test)
    weighted_f1 = new_weighted_f1_score(y_test, y_pred_test)
    f1 = f1_score(y_test > 0, y_pred_test > 0)
    rules, variable_count = descision_tree.get_clean_format(column_names_input)
    return weighted_f1, f1, variable_count, rules

def train_and_evaluate_decision_tree(layer : int, neuron : int, **kwargs):
    X_train, X_test, y_train, y_test = get_data(layer, neuron)
    descision_tree = train_decision_tree(X_train, y_train, layer, neuron, **kwargs)
    weighted_f1, f1, variable_count, rules = evaluate_decision_tree(descision_tree, X_test, y_test)
    return descision_tree, weighted_f1, f1, variable_count, rules

neurons = [0, 1, 2, 3, 4, 421]
min_impurity_decreases = [0.01, 0.001, 0.0001, 0.00001]

neurons = [2]
min_impurity_decreases = [0.0001]

for neuron in neurons:
    for min_impurity_decrease in min_impurity_decreases:
        print(f"Neuron {neuron} min_impurity_decrease {min_impurity_decrease}")
        descision_tree, weighted_f1, f1, variable_count, rules = train_and_evaluate_decision_tree(layer, neuron, min_impurity_decrease=min_impurity_decrease)
        print(f"Test f1 weighted: {weighted_f1}")
        print(f"Test f1: {f1}")
        results["neuron"].append(neuron)
        results["min_impurity_decrease"].append(min_impurity_decrease)
        results["f1_weighted"].append(weighted_f1)
        results["f1"].append(f1)
        results["CNF size"].append(variable_count)

        if variable_count <= 50:
            print("Rules:")
            print(rules)

results = pd.DataFrame(results)

Neuron 2 min_impurity_decrease 0.0001
Test f1 weighted: 0.960553424317472
Test f1: 0.8818737270875764


In [30]:
results

,neuron,min_impurity_decrease,f1_weighted,f1,CNF size
0,0,0.01000,0.000000,0.000000,1
1,0,0.00100,0.000000,0.000000,1
2,0,0.00010,0.645175,0.390738,37
3,0,0.00001,0.873408,0.681542,311
4,1,0.01000,0.000000,0.000000,1
5,1,0.00100,0.000000,0.000000,1
6,1,0.00010,0.815964,0.644571,25
7,1,0.00001,0.904599,0.764595,179
8,2,0.01000,0.000000,0.000000,1
9,2,0.00100,0.796542,0.544403,7


In [98]:
results_f1_weighted = results["f1_weighted"].values


numpy.ndarray

In [100]:
# round every float to 2 decimals
results = results.round(2)
results

,neuron,min_impurity_decrease,f1_weighted,f1,CNF size
0,0,0.01,0.00,0.00,1
1,0,0.00,0.00,0.00,1
2,0,0.00,0.65,0.39,37
3,0,0.00,0.87,0.68,311
4,1,0.01,0.00,0.00,1
5,1,0.00,0.00,0.00,1
6,1,0.00,0.82,0.64,25
7,1,0.00,0.90,0.76,179
8,2,0.01,0.00,0.00,1
9,2,0.00,0.80,0.54,7


In [ ]:
"""
Neuron 0 fuzzy_sparcity low
Train f1 wighted: [0.7021354]
Test f1 weighted: [0.64517511]
Train f1: 0.4029465930018416
Test f1: 0.3907380607814761
Train accuracy: 0.9797375
Test accuracy: 0.97895
Neuron 0 fuzzy_sparcity high
Train f1 wighted: [0.]
Test f1 weighted: [0.]
Train f1: 0.0
Test f1: 0.0
Train accuracy: 0.9747375
Test accuracy: 0.9738
['( => -0.0450)']
Neuron 1 fuzzy_sparcity low
Train f1 wighted: [0.8183083]
Test f1 weighted: [0.81596413]
Train f1: 0.6449451887941535
Test f1: 0.6445714285714286
Train accuracy: 0.985425
Test accuracy: 0.98445
Neuron 1 fuzzy_sparcity high
Train f1 wighted: [0.]
Test f1 weighted: [0.]
Train f1: 0.0
Test f1: 0.0
Train accuracy: 0.975525
Test accuracy: 0.9739
['( => -0.0645)']
Neuron 2 fuzzy_sparcity low
Train f1 wighted: [0.87310313]
Test f1 weighted: [0.85309672]
Train f1: 0.6448377581120944
Test f1: 0.6188118811881188
Train accuracy: 0.95485
Test accuracy: 0.9538
Neuron 2 fuzzy_sparcity high
Train f1 wighted: [0.79908459]
Test f1 weighted: [0.79654233]
Train f1: 0.5402757365805865
Test f1: 0.5444026612669945
Train accuracy: 0.9178875
Test accuracy: 0.92125
['(G4 not yours AND G4 not yours => -0.0176)', '(G4 not yours AND G4 yours => -0.1014)', '(G4 yours AND E6 not placed => 0.1332)', '(G4 yours AND E6 placed => 0.7973)']
Neuron 3 fuzzy_sparcity low
Train f1 wighted: [0.88659247]
Test f1 weighted: [0.88823854]
Train f1: 0.7624224201071662
Test f1: 0.7641135184620079
Train accuracy: 0.9229625
Test accuracy: 0.9227
Neuron 3 fuzzy_sparcity high
Train f1 wighted: [0.87005267]
Test f1 weighted: [0.87006652]
Train f1: 0.7576030373008729
Test f1: 0.7583965330444203
Train accuracy: 0.9225875
Test accuracy: 0.92195
['(A0 not yours => -0.0982)', '(A0 yours => 0.1340)']
Neuron 4 fuzzy_sparcity low
Train f1 wighted: [0.83555208]
Test f1 weighted: [0.84162394]
Train f1: 0.5831896551724138
Test f1: 0.5957095709570958
Train accuracy: 0.975825
Test accuracy: 0.9755
Neuron 4 fuzzy_sparcity high
Train f1 wighted: [0.]
Test f1 weighted: [0.]
Train f1: 0.0
Test f1: 0.0
Train accuracy: 0.9663125
Test accuracy: 0.9649
['( => -0.0653)']
Neuron 421 fuzzy_sparcity low
Train f1 wighted: [0.99963442]
Test f1 weighted: [0.9988347]
Train f1: 0.9902072420860851
Test f1: 0.9914691943127962
Train accuracy: 0.9994625
Test accuracy: 0.99955
Neuron 421 fuzzy_sparcity high
Train f1 wighted: [0.9959054]
Test f1 weighted: [0.99467223]
Train f1: 0.9811580882352942
Test f1: 0.9797882579403272
Train accuracy: 0.998975
Test accuracy: 0.99895
['(B4 not placed AND A4 not placed AND B4 not empty => -0.1152)', '(B4 not placed AND A4 not placed AND B4 empty => -0.0430)', '(B4 not placed AND A4 placed AND B4 not flipped => -0.0643)', '(B4 not placed AND A4 placed AND B4 flipped AND D1 not flipped => 2.0546)', '(B4 not placed AND A4 placed AND B4 flipped AND D1 flipped => 1.2594)', '(B4 placed AND C4 not empty AND B2 not flipped => 1.9735)', '(B4 placed AND C4 not empty AND B2 flipped => 1.3467)', '(B4 placed AND C4 empty => -0.1112)']
"""

In [61]:
# The f1 score from N421 is really good but the f1 score from N2 is not so good ...
# I couldn't test softmax and no softmax, rounding doesen't do much
threshold = 0
y_test_binary = y_test > threshold
y_pred_test_binary = y_pred_test > threshold
y_train_binary = y_train > threshold
y_pred_train_binary = y_pred_train > threshold

# get the MSE

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
print(mean_squared_error(y_test, y_pred_test), mean_squared_error(y_train, y_pred_train))
print(mean_absolute_error(y_test, y_pred_test), mean_absolute_error(y_train, y_pred_train))

print(precision_score(y_test_binary, y_pred_test_binary), precision_score(y_train_binary, y_pred_train_binary))
print(recall_score(y_test_binary, y_pred_test_binary), recall_score(y_train_binary, y_pred_train_binary))
print(f1_score(y_test_binary, y_pred_test_binary), f1_score(y_train_binary, y_pred_train_binary))

# evaluate(y_pred_test, y_test, 0)

0.011169868158954016 0.011069918091494232
0.046568370980348155 0.04661455556666943
0.6957328385899815 0.7069857697283312
0.5572065378900446 0.5927331887201736
0.6188118811881188 0.6448377581120944


In [59]:
descision_tree.get_clean_format(column_names = column_names_input)

['(G4 not yours AND G4 not yours => -0.0176)',
 '(G4 not yours AND G4 yours AND B4 not placed AND G4 not yours => -0.1154)',
 '(G4 not yours AND G4 yours AND B4 not placed AND G4 yours AND E2 not placed AND D7 not placed AND D1 not placed => -0.0728)',
 '(G4 not yours AND G4 yours AND B4 not placed AND G4 yours AND E2 not placed AND D7 not placed AND D1 placed => 0.2848)',
 '(G4 not yours AND G4 yours AND B4 not placed AND G4 yours AND E2 not placed AND D7 placed => 0.2406)',
 '(G4 not yours AND G4 yours AND B4 not placed AND G4 yours AND E2 placed => 0.5190)',
 '(G4 not yours AND G4 yours AND B4 placed AND G4 not yours => -0.0403)',
 '(G4 not yours AND G4 yours AND B4 placed AND G4 yours => 0.2309)',
 '(G4 yours AND E6 not placed AND E2 not placed AND D1 not placed AND H3 not yours AND D7 not placed AND G2 not placed AND G1 not placed AND C4 not placed AND B4 not placed AND G3 not placed AND C0 not placed AND H5 not yours => -0.0332)',
 '(G4 yours AND E6 not placed AND E2 not placed A

In [36]:
# Damn this operation is fucking slow
# I could do it on the gpu ...
# weighted_f1_score(y_pred_train > 0, y_train > 0), weighted_f1_score(y_pred_test > 0, y_test > 0)

In [37]:
dataset.columns[-2]

'H7 not_placed'

In [39]:
from sklearn.tree import export_text

# Assuming self.regressor is your trained DecisionTreeRegressor
tree_rules = export_text(descision_tree.regressor, feature_names=list(X.columns), decimals=10)
print(tree_rules)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [50]:
descision_tree.get_clean_format(column_names = column_names_input)

['(G4 not yours AND G4 not yours => -0.0176)',
 '(G4 not yours AND G4 yours => -0.1014)',
 '(G4 yours AND E6 placed => 0.7973)',
 '(G4 yours AND E6 not_placed => 0.1332)']

In [ ]:
dataset.columns

Index(['A0 empty_no_softmax', 'A0 yours_no_softmax', 'A0 mine_no_softmax',
       'A1 empty_no_softmax', 'A1 yours_no_softmax', 'A1 mine_no_softmax',
       'A2 empty_no_softmax', 'A2 yours_no_softmax', 'A2 mine_no_softmax',
       'A3 empty_no_softmax',
       ...
       'H3 not_placed', 'H4 placed', 'H4 not_placed', 'H5 placed',
       'H5 not_placed', 'H6 placed', 'H6 not_placed', 'H7 placed',
       'H7 not_placed', 'neuron activation'],
      dtype='object', length=898)

In [ ]:
2*(3 * 64 + 4 * 64) + 2
dataset["G4 empty_no_softmax"]

0        12.443144
1         1.525594
2        -6.436412
3        -6.848009
4        -8.937505
           ...    
99995    11.221716
99996    -8.789615
99997    -9.806160
99998   -14.874847
99999    -8.583365
Name: G4 empty_no_softmax, Length: 100000, dtype: float64

In [ ]:
# FAZIT
# Theorie das das Neuron große aktivierungen von G4 yours detected ist falsch. Runden von neuron outputs macht einen kelinen unterschied.
# TODO:
# Back to basics (mit focus cache arbeiten)
# Nochmal max activating examples anschauen ???
# Ist neuron aktiviert, wenn bestimmte andere neurons aktiviert sind ???
# Probes trainieren ???

In [ ]:
# N2 looks very hard on the visualization like it detects G4 Yours
# When the neuron activates highly, G4 is always yours
# sometimes when G4 is Yours the neuron doesen't activate
# apperantly also it can be sligthly activated when G4 is there This is more rare though and the activation is small
# I could imagine that we are missing inputs such as previous neurons activating, positions and other probes ...
# I could also imagine that this get's distorted by softmax (doesen't really make sense to use this nonlinearity, when the pre neuron activation is linear)
# It probably activates for large dot product with G4 yours, put the probe can also detect small cosine similiarities
# => Try out using the pre softmax activations
dataset_round = dataset.round(2)
# dataset_round = dataset_round[["G4 yours", "G4 yours_no_softmax", "G4 empty", "G4 mine", "G4 placed", "neuron activation"]]
dataset_round = dataset_round[["G4 yours", "G4 placed_no_softmax", "G4 yours_no_softmax", "G4 empty_no_softmax", "G4 mine_no_softmax", "neuron activation_no_softmax"]]
# dataset_round = dataset_round[dataset_round["neuron activation"] > 0]
dataset_round = dataset_round[dataset_round["G4 yours"] > 0.5]
dataset_round = dataset_round[dataset_round["neuron activation_no_softmax"] < 0]
# dataset_round = dataset_round[(dataset_round["neuron activation"] > 0) & (dataset_round["G4 yours"] < 0.5)]
dataset_round[:50]

,G4 yours,G4 placed_no_softmax,G4 yours_no_softmax,G4 empty_no_softmax,G4 mine_no_softmax,neuron activation_no_softmax
3,1.00,-13.26,9.13,-6.85,0.46,-0.16
8,1.00,-10.47,13.10,-8.44,6.97,-0.14
9,1.00,-10.75,10.84,-4.04,-1.96,-0.07
11,0.94,-12.10,6.97,-7.75,4.13,-0.17
12,1.00,-10.05,10.19,-8.01,3.17,-0.10
13,0.93,-13.52,6.88,-12.15,4.34,-0.17
16,0.94,-13.66,10.11,-9.67,7.30,-0.16
20,0.99,-11.07,10.45,-12.70,5.50,-0.14
21,1.00,-12.17,8.90,-9.54,0.72,-0.17
25,1.00,-13.10,9.32,-5.77,1.94,-0.16


In [ ]:
accuracies = [0.5, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
accuracies.reverse()
depths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
def scorefunction(depth, accuracy):
    return math.log(accuracy, 2) * 2**depth

scores = []
for accuracy in accuracies:
    scores += [[scorefunction(depth, accuracy) for depth in depths]]

# Turn this into a nice table
pd.DataFrame(scores, index=accuracies, columns=depths)

,1,2,3,4,5,6,7,8,9,10
0.99,-0.028999,-0.057998,-0.115997,-0.231993,-0.463986,-0.927972,-1.855945,-3.711890,-7.423780,-14.847559
0.95,-0.148001,-0.296002,-0.592005,-1.184009,-2.368019,-4.736037,-9.472074,-18.944149,-37.888298,-75.776595
0.90,-0.304006,-0.608012,-1.216025,-2.432049,-4.864099,-9.728198,-19.456396,-38.912792,-77.825584,-155.651168
0.85,-0.468931,-0.937861,-1.875722,-3.751444,-7.502888,-15.005776,-30.011552,-60.023105,-120.046210,-240.092420
0.80,-0.643856,-1.287712,-2.575425,-5.150850,-10.301699,-20.603398,-41.206796,-82.413592,-164.827185,-329.654369
0.75,-0.830075,-1.660150,-3.320300,-6.640600,-13.281200,-26.562400,-53.124800,-106.249600,-212.499200,-424.998399
0.50,-2.000000,-4.000000,-8.000000,-16.000000,-32.000000,-64.000000,-128.000000,-256.000000,-512.000000,-1024.000000


In [ ]:
# Print the Tree Rules
from sklearn.tree import export_text
r = export_text(regressor, feature_names=column_names_input)
print(r)
# (C4 Not Flipped AND A4 Placed) OR (C4 Flipped AND B4 Yours)
# (C4 Not Flipped AND A4 Placed) => High Activation, (C4 Flipped AND B4 Yours) => very High Activation, (C4 Flipped AND B4 Not Yours) => very Low Activation (This form is really nice)

NameError: name 'regressor' is not defined

In [ ]:
column_names_input = column_names_input.tolist()

A4_placed = column_names_input.index("A4 placed")
B4_flipped = column_names_input.index("B4 yours")
B4_placed = column_names_input.index("B4 placed")
C4_flipped = column_names_input.index("C4 yours")
# First Check Manually
# Create y_pred, such that it is 1 if (A4 Placed > 0.5 AND B4 Flipped > 0.5) OR (B4 Placed > 0.5 AND C4 Flipped > 0.5) otherwise -1
y_pred = np.zeros(y_test.shape)
for i in range(len(y_test)):
    if (X_test[i][A4_placed] > 0.5 and X_test[i][B4_flipped] > 0.5) or (X_test[i][B4_placed] > 0.5 and X_test[i][C4_flipped] > 0.5):
        y_pred[i] = 1
    else:
        y_pred[i] = -1

In [ ]:
# I'ts very weird, that B4 seems to be flipped and placed at the same time. This should not be possible.
# round the dataset to 2 decimal places
dataset = dataset.round(2)
dataset[["A4 placed", "B4 yours", "B4 placed", "C4 yours", "neuron activation"]][:50]

,A4 placed,B4 yours,B4 placed,C4 yours,neuron activation
0,0.0,0.02,0.0,1.00,-0.16
1,0.0,0.85,0.0,1.00,-0.14
2,0.0,0.00,0.0,0.00,-0.05
3,0.0,0.97,0.0,0.01,-0.15
4,0.0,0.00,0.0,0.00,-0.17
5,0.0,0.00,0.0,0.39,-0.05
6,0.0,0.00,0.0,0.01,-0.05
7,0.0,0.99,0.0,0.54,-0.12
8,0.0,0.00,0.0,0.00,-0.03
9,0.0,0.00,0.0,0.00,-0.17


In [ ]:
rows = 2
cols = 2
options = 2
batch = 1

probe_results_all = []
for probe_name in ["linear", "flipped", "placed"]:
    probe_result = t.randn((1, 2, 2, 2))
    for row in range(2):
        for col in range(2):
            for option in range(2):
                print(f"{tuple_to_label((row, col))} {get_direction_str(probe_name, option)}: {probe_result[0, row, col, option]}")
    probe_result = einops.rearrange(probe_result, "batch rows cols options -> batch (rows cols options)")
    probe_results_all.append(probe_result)
probe_results_all = t.cat(probe_results_all, dim=-1)

A0 empty: -0.33139568567276
A0 yours: 1.8200362920761108
A1 empty: 0.7885934710502625
A1 yours: -1.0049554109573364
B0 empty: -0.7153743505477905
B0 yours: -1.7870670557022095
B1 empty: -0.987899124622345
B1 yours: -1.0271559953689575
A0 flipped: 0.02292773686349392
A0 not_flipped: 1.3866825103759766
A1 flipped: 0.20812645554542542
A1 not_flipped: 1.0746756792068481
B0 flipped: -2.5803966522216797
B0 not_flipped: -0.8226088881492615
B1 flipped: -1.7223740816116333
B1 not_flipped: 1.0704987049102783
A0 placed: -2.397033452987671
A0 not_placed: 1.4333208799362183
A1 placed: 1.495888590812683
A1 not_placed: 1.9834816455841064
B0 placed: 0.6923328042030334
B0 not_placed: 0.0492352731525898
B1 placed: -0.09268636256456375
B1 not_placed: 0.23773331940174103


In [ ]:
probe_results_all

tensor([[-0.3314,  1.8200,  0.7886, -1.0050, -0.7154, -1.7871, -0.9879, -1.0272,
          0.0229,  1.3867,  0.2081,  1.0747, -2.5804, -0.8226, -1.7224,  1.0705,
         -2.3970,  1.4333,  1.4959,  1.9835,  0.6923,  0.0492, -0.0927,  0.2377]])

In [ ]:
probe_results_all

tensor([[-0.3314,  1.8200,  0.7886, -1.0050, -0.7154, -1.7871, -0.9879, -1.0272,
          0.0229,  1.3867,  0.2081,  1.0747, -2.5804, -0.8226, -1.7224,  1.0705,
         -2.3970,  1.4333,  1.4959,  1.9835,  0.6923,  0.0492, -0.0927,  0.2377]])

In [ ]:
get_accuracy(y_pred, y_test)

0.9986